In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
print(tf.__version__)

/Users/sreeharirammohan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [2]:
X = np.load('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/X.npy')
y = np.load('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/y.npy')

In [2]:
#Read in images from folder
DATA_PATH = "/Volumes/Sreehari 4TB backup/Lesions/"

classes = os.listdir(DATA_PATH)
del classes[0] #first entry is .DS Store not a directory

print(classes)

X = np.array([])
y = np.array([])

MAX_PHOTOS_PER_CLASS = 50

for class_name in classes:
    
    print("Working on class", class_name)

    class_folder = os.path.join(DATA_PATH,class_name)

    sub_files = os.listdir(class_folder)
    
    number_of_files = len(sub_files)
    i = 0
    
    for file in sub_files:
        if '.jpg' in file:
            image_filepath = os.path.join(class_folder, file)
            image_as_np_array = cv2.imread(image_filepath,flags=cv2.IMREAD_COLOR)
            
            #(400, 400, 3)
            #decrease the size of the image by 1/4 to 
            image_as_np_array = cv2.resize(image_as_np_array, (128,128), interpolation = cv2.INTER_LINEAR)
            
            #cv2.imshow('ImageWindow', image_as_np_array)
            #cv2.waitKey()
            
            X = np.append(X, image_as_np_array)
            y = np.append(y, class_name)

            print(str(i) + " - " + str(round((i/MAX_PHOTOS_PER_CLASS)*100, 2)) + "%", end='\r', flush=True)

            i += 1
            
            if (i > MAX_PHOTOS_PER_CLASS):
                break
                
        


['Actinic_Keratosis', 'Basel_Cell_Carcinoma', 'Melanoma', 'Seaborrheic Keratosis']
Working on class Actinic_Keratosis
Working on class Basel_Cell_Carcinoma
Working on class Melanoma
Working on class Seaborrheic Keratosis


In [3]:
X = X.reshape(-1,64,64,3)
X.shape

(204, 64, 64, 3)

In [4]:
y.shape

(204,)

In [5]:
#Normalize image array values between 0-1
X /= 255.0

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [7]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [8]:
np.save('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/X_128_128.npy', X)
np.save('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/y_128_128.npy', y)

In [10]:
print(label_encoder.inverse_transform([0, 1, 2, 3]))

['Actinic_Keratosis' 'Basel_Cell_Carcinoma' 'Melanoma'
 'Seaborrheic Keratosis']


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
X_train.shape

(406, 400, 400, 3)

In [5]:
X_test.shape

(200, 400, 400, 3)

In [6]:
y_train.shape

(406,)

In [7]:
y_test.shape

(200,)

In [8]:
print(y_test)

[5 2 0 0 1 5 0 1 4 0 0 5 4 2 0 3 0 2 1 5 0 1 5 2 1 1 3 2 5 4 0 3 4 5 2 5 5
 3 2 2 5 4 0 5 5 0 2 0 5 4 5 1 5 5 5 5 1 2 2 1 4 1 0 0 1 5 1 5 3 0 2 5 0 1
 1 2 1 2 1 2 1 4 1 4 3 0 0 4 2 2 1 4 4 0 3 4 2 4 0 5 0 0 2 3 3 0 2 5 2 1 4
 2 1 5 1 3 2 2 3 4 3 2 0 2 5 3 1 3 2 5 2 0 1 4 1 5 3 0 0 3 0 0 1 3 0 5 1 5
 4 0 0 5 5 3 3 2 2 0 4 1 0 3 2 0 4 5 1 2 2 5 3 3 3 5 4 1 0 4 0 0 2 1 5 5 4
 4 3 4 2 3 5 2 4 0 2 4 5 3 1 4]


In [9]:
import keras 
from keras.models import Sequential
from keras.layers import *

num_classes = 6
input_shape=(400,400,3)
epochs = 10
batch_size = 32

Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 406 samples, validate on 200 samples
Epoch 1/10
406/406 [==============================] - 5715s 14s/step - loss: 2.0473 - acc: 0.1576 - val_loss: 1.7816 - val_acc: 0.1400
Epoch 2/10
406/406 [==============================] - 5499s 14s/step - loss: 1.7669 - acc: 0.2488 - val_loss: 1.7487 - val_acc: 0.2650
Epoch 3/10
406/406 [==============================] - 5649s 14s/step - loss: 1.6936 - acc: 0.3030 - val_loss: 2.0219 - val_acc: 0.1700
Epoch 4/10
406/406 [==============================] - 5606s 14s/step - loss: 1.7842 - acc: 0.2463 - val_loss: 1.7327 - val_acc: 0.3100
Epoch 5/10
406/406 [==============================] - 5614s 14s/step - loss: 1.6368 - acc: 0.2808 - val_loss: 1.6517 - val_acc: 0.2550
Epoch 6/10
406/406 [==============================] - 5619s 14s/step - loss: 1.5283 - acc: 0.3424 - val_loss: 1.5862 - val_acc: 0.3300
Epoch 7/10
406/406 [==============================] - 5897s 15s/step - loss: 1.5928 - acc: 0.3374 - val_loss: 1.6013 - val_acc: 0.3150
Epoch 8/1

In [13]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.4368384075164795
Test accuracy: 0.42


In [15]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    tf.keras.models.save_model(
        model,
        '/Volumes/Sreehari 4TB backup/CES-Model_and_Data/model.h5',
        overwrite=True,
        include_optimizer=True
    )

In [25]:
tf.VERSION

'1.10.0'

In [24]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file("/Volumes/Sreehari 4TB backup/CES-Model_and_Data/model.h5")
tflite_model = converter.convert()
open('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/model_output.tflite', "wb").write(tflite_model)

INFO:tensorflow:Froze 8 variables.


ValueError: Cannot create a tensor proto whose content is larger than 2GB.

In [18]:
converter = tf.contrib.lite.TocoConverter.from_keras_model_file('/Volumes/Sreehari 4TB backup/CES-Model_and_Data/model.h5')
converter.post_training_quantize = True
converter.quantize_weights = True
tflite_model = converter.convert()
open("/Volumes/Sreehari 4TB backup/CES-Model_and_Data/quantized_model.tflite", "wb").write(tflite_model)

KeyboardInterrupt: 

In [ ]:
print("done")